## Test

In [13]:
from keras.datasets import mnist
from keras.models import Model, Sequential
from keras.layers import Input, Flatten, Dense, Reshape
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import BatchNormalization, Activation
from keras.optimizers import Adam

import numpy as np

import matplotlib.pyplot as plt

import os

In [11]:
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)
latent_dim = 100

optimizer = Adam(0.0002, 0.5)

discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy',
                     optimizer=optimizer,
                     metrics=['accuracy'])

generator = build_generator()

z = Input(shape=(latent_dim,))
img = generator(z)

discriminator.trainable = False

validity = discriminator(img)

combined = Model(z, validity)
combined.compile(loss='binary_crossentropy',
                optimizer=optimizer)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________
____

In [4]:
def build_generator():
    
    model = Sequential()
    
    model.add(Dense(256, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(np.prod(img_shape), activation='tanh'))
    model.add(Reshape(img_shape))
    
    model.summary()
    
    noise = Input(shape=(latent_dim,))
    img = model(noise)
    
    return Model(noise, img)

In [9]:
def build_discriminator():
    
    model = Sequential()
    
    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    
    model.summary()
    
    img = Input(shape=img_shape)
    validity = model(img)
    
    return Model(img, validity)

In [6]:
def train(epochs, batch_size=128, sample_interval=50):
    
    (X_train, _), (_, _) = mnist.load_data()
    
    X_train = X_train / 127.5 - 1
    X_train = np.expand_dims(X_train, axis=3)
    
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))
    
    for epoch in range(epochs+1):
        
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]
        
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        gen_imgs = generator.predict(noise)
        
        d_loss_real = discriminator.train_on_batch(imgs, valid)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        
        g_loss = combined.train_on_batch(noise, valid)
        
        print("%d [D Loss: %f, acc: %.2f%%] [G Loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
        
        if epoch % sample_interval == 0:
            sample_images(epoch)

In [16]:
def sample_images(epoch):
    r, c = 5, 5
    
    noise = np.random.normal(0, 1, (r*c, latent_dim))
    gen_imgs = generator.predict(noise)
    
    gen_imgs = 0.5 * gen_imgs + 0.5
    
    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("test_imgs/%d.png" % epoch)
    plt.close()

In [17]:
train(epochs=200, batch_size=32)

0 [D Loss: 0.019507, acc: 100.00%] [G Loss: 3.795707]
1 [D Loss: 0.023905, acc: 100.00%] [G Loss: 3.839781]
2 [D Loss: 0.018352, acc: 100.00%] [G Loss: 3.961220]
3 [D Loss: 0.019144, acc: 100.00%] [G Loss: 3.885123]
4 [D Loss: 0.020809, acc: 100.00%] [G Loss: 3.953260]
5 [D Loss: 0.022088, acc: 100.00%] [G Loss: 4.006546]
6 [D Loss: 0.014083, acc: 100.00%] [G Loss: 4.031678]
7 [D Loss: 0.018892, acc: 100.00%] [G Loss: 3.932746]
8 [D Loss: 0.019613, acc: 100.00%] [G Loss: 3.977071]
9 [D Loss: 0.008578, acc: 100.00%] [G Loss: 3.958097]
10 [D Loss: 0.012147, acc: 100.00%] [G Loss: 4.059774]
11 [D Loss: 0.014657, acc: 100.00%] [G Loss: 4.217986]
12 [D Loss: 0.016071, acc: 100.00%] [G Loss: 4.021207]
13 [D Loss: 0.010657, acc: 100.00%] [G Loss: 4.008115]
14 [D Loss: 0.014696, acc: 100.00%] [G Loss: 4.178343]
15 [D Loss: 0.016166, acc: 100.00%] [G Loss: 4.149767]
16 [D Loss: 0.017569, acc: 100.00%] [G Loss: 3.989317]
17 [D Loss: 0.015510, acc: 100.00%] [G Loss: 4.259027]
18 [D Loss: 0.015315

151 [D Loss: 0.574423, acc: 75.00%] [G Loss: 2.571733]
152 [D Loss: 0.198068, acc: 92.19%] [G Loss: 3.853209]
153 [D Loss: 0.620753, acc: 67.19%] [G Loss: 2.704077]
154 [D Loss: 0.175304, acc: 96.88%] [G Loss: 3.132883]
155 [D Loss: 0.412401, acc: 75.00%] [G Loss: 2.228769]
156 [D Loss: 0.216434, acc: 95.31%] [G Loss: 3.522728]
157 [D Loss: 0.335551, acc: 87.50%] [G Loss: 2.560954]
158 [D Loss: 0.250460, acc: 92.19%] [G Loss: 3.269819]
159 [D Loss: 0.742354, acc: 64.06%] [G Loss: 2.003261]
160 [D Loss: 0.177735, acc: 95.31%] [G Loss: 3.034252]
161 [D Loss: 0.210171, acc: 96.88%] [G Loss: 3.228730]
162 [D Loss: 0.417912, acc: 75.00%] [G Loss: 2.895365]
163 [D Loss: 0.277010, acc: 93.75%] [G Loss: 3.024930]
164 [D Loss: 0.603008, acc: 67.19%] [G Loss: 2.469289]
165 [D Loss: 0.210628, acc: 89.06%] [G Loss: 3.604887]
166 [D Loss: 0.420448, acc: 78.12%] [G Loss: 3.171346]
167 [D Loss: 0.275304, acc: 93.75%] [G Loss: 2.862654]
168 [D Loss: 0.339528, acc: 87.50%] [G Loss: 3.369945]
169 [D Los